In [1]:
import pickle
import import_ipynb
import Week2
from nltk.tokenize import word_tokenize

importing Jupyter notebook from Week2.ipynb


In [2]:
index = None
vocab = None
doc_lens = None
with open('vocab','rb') as fp:
    vocab = pickle.load(fp)
with open('index','rb') as fp:
    index = pickle.load(fp)
with open('doc_lens', 'rb') as fp:
    doc_lens = pickle.load(fp)

In [3]:
index['project']

[[88], {0: 3, 3: 1, 9: 84}]

In [4]:
print(len(vocab))
print(len(index.keys()))
print(doc_lens)

5431
5431
[5963, 1838, 1064, 1592, 2062, 5056, 6460, 1704, 5073, 3881]


In [5]:
import numpy as np
matrix = np.zeros((10, len(vocab)))
print(matrix)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
def find_tf():
    for key in index:
        term = index[key]
        for tf in term[1].keys():
            term_freq = term[1][tf]
            i = vocab.index(key)
            matrix[tf][i] = term_freq/doc_lens[tf]

In [7]:
find_tf()
print(matrix)
idf_matrix = np.zeros(len(vocab))
print(idf_matrix)

[[0.0005031  0.0005031  0.0008385  ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.02164391 0.00695697 0.00334965 ... 0.00025767 0.00025767 0.00025767]]
[0. 0. 0. ... 0. 0. 0.]


In [8]:
def find_idf():
    for key in index:
        term = index[key]
        idf = len(term[1].keys())
        res = np.log10(10/idf)
        i = vocab.index(key)
        idf_matrix[i] = res

In [9]:
find_idf()

In [10]:
tf_idf = matrix*idf_matrix
print(tf_idf)

[[0.00026306 0.00035165 0.00058609 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01131714 0.00486271 0.00234131 ... 0.00025767 0.00025767 0.00025767]]


In [11]:
from collections import Counter
def query_tf_idf(query):
    counter = Counter(query)
    wc = len(query)
    tfm = np.zeros(len(vocab))
    idfm = np.zeros(len(vocab))
    for token in np.unique(query):
        tf = counter[token]/wc
        i = vocab.index(token)
        idfm = idf_matrix[i]
        tfm[i] = tf
    return(tfm*idfm)

In [12]:
query = "A somewhat higher level on the inclined plane is illustrated by what are called tropisms obligatory movements which the animal makes, adjusting its whole body so that physiological equilibrium results in relation to gravity, pressure, currents, moisture, heat, light, electricity, and surfaces of contact. A moth is flying past a candle; the eye next the light is more illumined than the other; a physiological inequilibrium results, affecting nerve-cells and muscle-cells; the outcome is that the moth automatically adjusts its flight so that both eyes become equally illumined; in doing this it often flies into the candle."
query = Week2.query(query)
query = word_tokenize(query)
query_matrix = query_tf_idf(query)

In [13]:
ranking = []
for i in range(0, 10):
    row = tf_idf[i]
    rank = np.dot(row, np.transpose(query_matrix))/(np.linalg.norm(row)*np.linalg.norm(query_matrix))
    ranking.append([rank, i + 1])

In [14]:
ranking.sort(reverse = True)
print(ranking)

[[0.17095819993793498, 6], [0.08749999946608858, 7], [0.059803234253092995, 9], [0.04263802915754703, 2], [0.03772075156734901, 1], [0.027447933174219816, 5], [0.024697211710131783, 4], [0.020657048627909905, 8], [0.019254623436106455, 3], [0.01511071480789552, 10]]


In [25]:
ranking = []
x_matrix = tf_idf[2]
for i in range(0, 10):
    if(i!= 2):
        row = tf_idf[i]
        rank = np.dot(row, np.transpose(x_matrix))/(np.linalg.norm(row)*np.linalg.norm(x_matrix))
        ranking.append([rank])

In [26]:
ranking.sort(reverse = True)
print(ranking)

[[0.13426602493169734], [0.13278954182099384], [0.10593716973328518], [0.07770672542940116], [0.04901278829310735], [0.045866021419684655], [0.03926671172575528], [0.036671773303474896], [0.02510436278823775]]
